In [ ]:
# Install local biota to overwrite system installation

In [ ]:
%%capture
!pip install biota/

In [ ]:
from functools import partial

import ipyvuetify as v
from sepal_ui import mapping as m
from sepal_ui import sepalwidgets as sw

from component.widget.custom_widgets import *
from component.scripts.scripts import *
from component.message import cm
from component.tiles import *

In [ ]:
# Map
map_ = m.SepalMap()
parameters = Parameters(map_=map_)
map_.on_interaction(partial(return_coordinates, parameters=parameters, map_=map_))

In [ ]:
process = Process(parameters)

In [ ]:
process_tile = sw.Tile(id_='process', title='Process', inputs=[process])

parameters_tile = sw.Tile(id_='parameters', title='Parameters', inputs=[parameters])

appBar = sw.AppBar('SMFM Biota')

content = [
    parameters_tile,
    process_tile,
]

#create a drawer 
item_param = sw.DrawerItem('Parameters', 'mdi-map-marker-check', card="parameters").display_tile(content)
item_process = sw.DrawerItem('Process', 'mdi-earth', card="process").display_tile(content)

code_link = 'https://github.com/ingdanielguerrero/smfm_biota'
wiki_link = 'https://github.com/ingdanielguerrero/smfm_biota/blob/main/README.md'
issue = 'https://github.com/ingdanielguerrero/smfm_biota/issues/new'

items = [
    item_param,
    item_process,
]

drawer = sw.NavDrawer(items, code = code_link, wiki = wiki_link, issue = issue).display_drawer(appBar.toggle_button)

#build the app 
#
app = sw.App(
    appBar = appBar,
    tiles=content, 
    navDrawer=drawer
).show_tile('parameters')
#display the app
app